# Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from contextlib import redirect_stdout
import numpy as np
import os
import pandas as pd
import plotly.express as px

from cost import *
from plotting import *
from prices import *
from imputation import *
from inflation import *
from regression import *
from utils import *

In [3]:
estimation_method = 'hardware-capex-opex'
compute_threshold_method = 'top_n'
compute_threshold = 10
variant = 'no-TPU'

estimation_method_lookup = {
    'cloud': estimate_cloud_costs,
    'amortized': estimate_amortized_hardware_costs,
    'hardware-acquisition': estimate_hardware_acquisition_cost,
    'hardware-capex-opex': estimate_hardware_capex_opex,
}
cost_estimation_function = estimation_method_lookup[estimation_method]

results_dir = f'results/{estimation_method}-{compute_threshold_method}={compute_threshold}-{variant}/'
os.makedirs(results_dir, exist_ok=True)

# Load data

In [4]:
frontier_pcd_df, hardware_df, price_df = load_data_for_cost_estimation(
    compute_threshold_method=compute_threshold_method, compute_threshold=compute_threshold,
)

In [5]:
len(frontier_pcd_df), len(hardware_df), len(price_df)

(67, 5510, 142)

# Cost estimation

In [6]:
with open(f'{results_dir}/cost_estimation.out', 'w') as f:
    with redirect_stdout(f):
        cost_df = cost_estimation_function(frontier_pcd_df, hardware_df, price_df)

In [7]:
cost_df

,System,Domain,Task,Authors,Notability criteria,Notability criteria notes,Model accessibility,Link,Citations,Reference,...,Foundation model,Training compute lower bound,Training compute upper bound,Training chip-hours,Code accessibility,Dataset accessibility,Accessibility notes,Organization categorization (from Organization),Possibly over 1e23 FLOP,Cost
75,Gemini Ultra,Multimodal,"Language modelling,Visual question answering,C...",Gemini Team,SOTA improvement,""" Evaluation on a broad range of benchmarks sh...",Hosted access (no API),https://storage.googleapis.com/deepmind-media/...,633.0,Gemini: A Family of Highly Capable Multimodal ...,...,NaN,NaN,NaN,132000000.0,NaN,NaN,NaN,Industry,NaN,NaN
91,Inflection-2,Language,Language modelling,NaN,Significant use,Inflection-2 either already powers Pi or soon ...,Hosted access (no API),https://inflection.ai/inflection-2,NaN,Inflection-2: The Next Step Up,...,checked,NaN,NaN,NaN,NaN,NaN,"via Pi, no API",Industry,NaN,1.160021e+07
116,Grok-1,Language,"Language modelling,Chat",NaN,SOTA improvement,"""On these benchmarks, Grok-1 displayed strong ...",Open source,"https://x.ai/model-card/, https://x.ai/blog/gr...",NaN,Announcing Grok,...,checked,2,7.0,NaN,Unreleased,Unreleased,apache 2.0,Industry,checked,NaN
130,ChatGLM3,Multimodal,"Chat,Visual question answering",NaN,SOTA improvement,"Aiming at GPT-4V, ChatGLM3 has implemented ite...",NaN,https://www.zhipuai.cn/en/news/76,NaN,Zhipu AI launches third-generation base model,...,checked,NaN,NaN,NaN,NaN,NaN,NaN,Industry,NaN,NaN
176,Falcon-180B,Language,Language modelling,"Ebtesam Almazrouei, Hamza Alobeidli, Abdulaziz...",SOTA improvement,"""It's currently at the top of the Hugging Face...",Open access (restricted use),https://falconllm.tii.ae/falcon-180b.html; htt...,86.0,The Falcon Series of Open Language Models,...,NaN,NaN,NaN,17694720.0,NaN,NaN,"""Falcon 180b can be commercially used but unde...",Government,NaN,3.543159e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1145,Xception,Vision,Image classification,François Chollet,Highly cited,NaN,NaN,https://arxiv.org/abs/1610.02357,11578.0,Xception: Deep Learning with Depthwise Separab...,...,NaN,NaN,NaN,43200.0,NaN,NaN,NaN,Industry,NaN,2.623551e+04
1146,GNMT,Language,Translation,"Yonghui Wu, Mike Schuster, Zhifeng Chen, Quoc ...",Highly cited,NaN,Hosted access (no API),https://arxiv.org/abs/1609.08144,6196.0,Google's Neural Machine Translation System: Br...,...,NaN,NaN,NaN,414720.0,NaN,NaN,presumably deployed via Google translate,Industry,NaN,1.612009e+05
1182,AlphaGo Lee,Games,Go,"David Silver, Aja Huang, Chris J. Maddison, Ar...",Highly cited,NaN,NaN,https://www.nature.com/articles/nature16961,14887.0,Mastering the game of Go with deep neural netw...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Industry,NaN,NaN
1186,ResNet-152 (ImageNet),Vision,Image classification,"Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun",Highly cited,NaN,NaN,https://arxiv.org/abs/1512.03385,156882.0,Deep Residual Learning for Image Recognition,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Industry,NaN,NaN


In [8]:
cost_df['Cost'].notna().sum()

25

Use the below to check data availability for specific systems

In [9]:
# system = 'WizardLM-7B'
# row = cost_df.loc[cost_df['System'] == system]
# print('Cost:', row['Cost'].values[0])
# print('Training hardware:', row['Training hardware'].values[0])
# print('Training time (hours):', row['Training time (hours)'].values[0])
# print('Hardware quantity:', row['Hardware quantity'].values[0])
# print('Hardware utilization:', row['Hardware utilization'].values[0])

# Apply inflation adjustment

In [10]:
cost_df['Cost'].dropna()

91      1.160021e+07
176     3.543159e+07
215     2.328665e+06
227     6.015989e+06
309     1.141358e+08
325     2.284406e+06
369     1.226792e+07
380     1.463166e+06
420     3.134881e+06
421     1.329454e+06
485     1.096406e+06
590     6.858062e+06
597     8.430421e+05
703     1.915049e+05
765     2.207634e+05
772     4.009866e+06
807     1.046808e+05
857     3.412142e+05
858     1.891154e+05
883     1.383296e+05
1092    4.376435e+04
1134    7.097896e+02
1145    2.623551e+04
1146    1.612009e+05
1188    2.418115e+02
Name: Cost, dtype: float64

In [11]:
cost_df = adjust_column_for_inflation(cost_df, 'Cost', 'data/PCU518210518210.csv', '2023-12-01')

In [12]:
cost_df['Cost (inflation-adjusted)'].dropna()

91      1.162737e+07
176     3.565113e+07
215     2.341945e+06
227     6.050299e+06
309     1.153608e+08
325     2.313581e+06
369     1.251457e+07
380     1.492584e+06
420     3.197416e+06
421     1.355974e+06
485     1.113754e+06
590     7.074952e+06
597     8.712443e+05
703     1.987343e+05
765     2.303021e+05
772     4.208945e+06
807     1.097811e+05
857     3.639313e+05
858     2.017062e+05
883     1.488732e+05
1092    4.727106e+04
1134    7.751033e+02
1145    2.864966e+04
1146    1.760344e+05
1188    2.689983e+02
Name: Cost (inflation-adjusted), dtype: float64

In [13]:
assert cost_df['Cost (inflation-adjusted)'].notna().sum() == cost_df['Cost'].notna().sum()

# Regression

In [14]:
cost_df['Publication date (float)'] = datetime_to_float_year(pd.to_datetime(cost_df['Publication date']))

In [15]:
reg_results = fit_ols_regression(cost_df, ['Publication date (float)'], 'Cost (inflation-adjusted)', logy=True)
reg_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.754
Model:                            OLS   Adj. R-squared:                  0.743
Method:                 Least Squares   F-statistic:                     70.49
Date:                Mon, 06 May 2024   Prob (F-statistic):           1.85e-08
Time:                        19:44:07   Log-Likelihood:                -24.008
No. Observations:                  25   AIC:                             52.02
Df Residuals:                      23   BIC:                             54.45
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -905.8836    108.588     -8.342      0.000   -1130.515    -681.252
x1             0.4511      0.054      8.396      0.000       0.340       0.562
==============================================================================
Omnibus:                        0.801   Durbin-Watson:                   2.654
Prob(Omnibus):                  0.670   Jarque-Bera (JB):                0.739
Skew:                           0.372   Prob(JB):                        0.691
Kurtosis:                       2.606   Cond. No.                     1.66e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.66e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [16]:
with open(f'{results_dir}/regression_results.out', 'w') as f:
    with redirect_stdout(f):
        print_growth_rates(reg_results)
print_growth_rates(reg_results)

R^2: 0.75
0.45 OOMs/year (95% CI: 0.34, 0.56)
2.8x/year (95% CI: 2.2x, 3.6x)
doubling time of 8 months (95% CI: 6, 11)


In [17]:
pred_start_year = 2015
pred_end_year = 2025
pred_start_date = f'{pred_start_year}-01-01'
pred_end_date = f'{pred_end_year}-01-01'

pred_years = pd.DataFrame({'Publication date (float)': np.linspace(pred_start_year, pred_end_year, 100)})
pred_years

,Publication date (float)
0,2015.00000
1,2015.10101
2,2015.20202
3,2015.30303
4,2015.40404
...,...
95,2024.59596
96,2024.69697
97,2024.79798
98,2024.89899


In [18]:

predicted_cost_df = get_predictions(reg_results, pred_years, ['Publication date (float)'])
predicted_cost_df['Publication date'] = predicted_cost_df['Publication date (float)'].apply(float_year_to_datetime)
predicted_cost_df

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper,Publication date (float),Publication date
0,3.138329,0.343338,2.428081,3.848577,1.601018,4.675640,2015.00000,2015-01-01
1,3.183897,0.338333,2.484003,3.883792,1.651342,4.716452,2015.10101,2015-02-06
2,3.229466,0.333341,2.539898,3.919033,1.701599,4.757332,2015.20202,2015-03-15
3,3.275034,0.328363,2.595765,3.954304,1.751788,4.798281,2015.30303,2015-04-21
4,3.320603,0.323399,2.651601,3.989604,1.801907,4.839298,2015.40404,2015-05-28
...,...,...,...,...,...,...,...,...
95,7.467330,0.238354,6.974258,7.960402,6.017505,8.917155,2024.59596,2024-08-06
96,7.512898,0.242894,7.010434,8.015363,6.059852,8.965945,2024.69697,2024-09-12
97,7.558467,0.247470,7.046535,8.070399,6.102120,9.014814,2024.79798,2024-10-19
98,7.604035,0.252081,7.082567,8.125504,6.144309,9.063762,2024.89899,2024-11-25


# Plots

In [19]:
if compute_threshold_method == 'window_percentile':
    title_suffix = f' to train top {100 - compute_threshold}% most compute-intensive ML models'
elif compute_threshold_method == 'top_n':
    title_suffix = f' to train the running top-{compute_threshold} most compute-intensive ML models'

plot_title_lookup = {
    'cloud': 'Cloud compute cost' + title_suffix,
    'amortized': 'Amortized cost of hardware' + title_suffix,
    'hardware-acquisition': 'Acquisition cost of hardware' + title_suffix,
    'hardware-capex-opex': 'Hardware CapEx + OpEx' + title_suffix,
}

In [20]:
fig = px.scatter(
    cost_df,
    x='Publication date',
    y='Cost (inflation-adjusted)',
    text='System',
    log_y=True,
)
fig.update_traces(textposition='top center')

# no legend
fig.update_layout(showlegend=False)

# axis labels
fig.update_xaxes(title_text='Publication date')
fig.update_yaxes(title_text='Cost (2023 USD)')

# title
fig.update_layout(title_text=plot_title_lookup[estimation_method])

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    title_font=dict(
        size=16,
    )
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# axis limits
fig.update_xaxes(range=['2015-01-01', '2025-01-01'])

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'cost_scatter')

fig.show()

In [21]:
label_systems = ['GNMT', 'AlphaGo Master', 'AlphaGo Zero', 'AlphaZero', 'DALL-E', 'GPT-3 175B (davinci)', 'PaLM (540B)', 'Llama 2-70B', 'Falcon 180B', 'GPT-4', 'Gemini Ultra', 'Inflection-2']

tpu_mask = cost_df['Training hardware'].str.contains('TPU', na=False)
tpu_cost_df = cost_df.loc[tpu_mask]
gpu_cost_df = cost_df.loc[~tpu_mask]

fig = px.scatter(
    gpu_cost_df,
    x='Publication date',
    y='Cost (inflation-adjusted)',
    log_y=True,
)
# fig.add_scatter(
#     x=tpu_cost_df['Publication date'],
#     y=tpu_cost_df['Cost (inflation-adjusted)'],
#     mode='markers',
#     marker_symbol='circle-open' if estimation_method != 'cloud' else 'circle',
#     name='Using equivalent price of TPU' if estimation_method != 'cloud' else '',
#     showlegend=estimation_method != 'cloud',
# )
fig.add_scatter(
    x=gpu_cost_df.loc[gpu_cost_df['System'].isin(label_systems)]['Publication date'],
    y=gpu_cost_df.loc[gpu_cost_df['System'].isin(label_systems)]['Cost (inflation-adjusted)'],
    text=gpu_cost_df.loc[gpu_cost_df['System'].isin(label_systems)]['System'],
    mode='text',
    showlegend=False,
)
# fig.add_scatter(
#     x=tpu_cost_df.loc[tpu_cost_df['System'].isin(label_systems)]['Publication date'],
#     y=tpu_cost_df.loc[tpu_cost_df['System'].isin(label_systems)]['Cost (inflation-adjusted)'],
#     text=tpu_cost_df.loc[tpu_cost_df['System'].isin(label_systems)]['System'],
#     mode='text',
#     showlegend=False,
# )

# Marker color
fig.update_traces(
    marker=dict(
        color='rgb(0,100,200)',
    ),
    selector=dict(mode='markers'),
)

# Shade in CI
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean_ci_lower'],
    mode='lines',
    line=dict(width=0),
    showlegend=False,
)
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean_ci_upper'],
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(0,100,200,0.2)',
    line=dict(width=0),
    name='95% CI of mean',
)
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean'],
    mode='lines',
    line=dict(color='rgb(0,100,200)'),
    name=f'Regression mean (growth rate: {10**reg_results.params[1]:.1f}x per year)',
)

fig.update_traces(textposition='top center')

# axis limits
fig.update_xaxes(range=['2015-01-01', '2025-01-01'])
fig.update_yaxes(range=[1, 9])

# legend on bottom-right of the axes
fig.update_layout(
    legend=dict(
        x=0.45,
        y=0.05,
    )
)

# axis labels
fig.update_xaxes(title_text='Publication date')
fig.update_yaxes(title_text='Cost (2023 USD)')

# title
fig.update_layout(title_text=plot_title_lookup[estimation_method])

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    title_font=dict(
        size=16,
    )
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# axis limits
fig.update_xaxes(range=[pred_start_date, pred_end_date])

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'cost_regression')

fig.show()

# Export data

In [22]:
keep_cols = [
    'System',
    'Domain',
    'Task',
    'Model accessibility',
    'Reference',
    'Publication date',
    'Organization',
    'Parameters',
    'Training compute (FLOP)',
    'Training dataset size (datapoints)',
    'Epochs',
    'Training time (hours)',
    'Training hardware',
    'Country (from Organization)',
    'Base model',
    'Finetune compute (FLOP)',
    'Hardware quantity',
    'Hardware utilization',
    'Training cloud compute vendor',
    'Training data center',
    # 'Training time (chip hours)',
    'Cost',
    'Cost (inflation-adjusted)',
]
cost_df[keep_cols]

,System,Domain,Task,Model accessibility,Reference,Publication date,Organization,Parameters,Training compute (FLOP),Training dataset size (datapoints),...,Training hardware,Country (from Organization),Base model,Finetune compute (FLOP),Hardware quantity,Hardware utilization,Training cloud compute vendor,Training data center,Cost,Cost (inflation-adjusted)
75,Gemini Ultra,Multimodal,"Language modelling,Visual question answering,C...",Hosted access (no API),Gemini: A Family of Highly Capable Multimodal ...,2023-12-06,Google DeepMind,NaN,5.000000e+25,NaN,...,Google TPU v4,Multinational,NaN,NaN,55000.0,NaN,NaN,NaN,NaN,NaN
91,Inflection-2,Language,Language modelling,Hosted access (no API),Inflection-2: The Next Step Up,2023-11-22,Inflection AI,NaN,1.001000e+25,NaN,...,NVIDIA H100 SXM5,United States of America,NaN,NaN,5000.0,NaN,NaN,NaN,1.160021e+07,1.162737e+07
116,Grok-1,Language,"Language modelling,Chat",Open source,Announcing Grok,2023-11-04,xAI,3.140000e+11,2.900000e+24,NaN,...,NaN,United States of America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130,ChatGLM3,Multimodal,"Chat,Visual question answering",NaN,Zhipu AI launches third-generation base model,2023-10-27,Zhipu AI,1.300000e+11,1.092000e+24,1.050000e+12,...,NaN,China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
176,Falcon-180B,Language,Language modelling,Open access (restricted use),The Falcon Series of Open Language Models,2023-09-06,Technology Innovation Institute,1.800000e+11,3.760000e+24,2.625000e+12,...,NVIDIA A100 SXM4 40 GB,United Arab Emirates,NaN,NaN,4096.0,0.1876,Amazon Web Services,NaN,3.543159e+07,3.565113e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1145,Xception,Vision,Image classification,NaN,Xception: Deep Learning with Depthwise Separab...,2016-10-07,Google,2.285595e+07,4.360000e+20,3.500000e+08,...,NVIDIA Tesla K80,United States of America,NaN,NaN,60.0,NaN,NaN,NaN,2.623551e+04,2.864966e+04
1146,GNMT,Language,Translation,Hosted access (no API),Google's Neural Machine Translation System: Br...,2016-09-26,Google,2.780000e+08,6.900000e+21,3.600000e+08,...,NVIDIA Tesla K80,United States of America,NaN,NaN,96.0,NaN,NaN,NaN,1.612009e+05,1.760344e+05
1182,AlphaGo Lee,Games,Go,NaN,Mastering the game of Go with deep neural netw...,2016-01-27,DeepMind,NaN,1.900000e+21,2.940000e+07,...,NaN,United Kingdom of Great Britain and Northern I...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1186,ResNet-152 (ImageNet),Vision,Image classification,NaN,Deep Residual Learning for Image Recognition,2015-12-10,Microsoft,6.000000e+07,1.210000e+19,1.280000e+06,...,NaN,United States of America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
cost_df[keep_cols].to_csv(results_dir + 'price dataset.csv', index=False)